# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from stock_prices_predictions.get_data import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import layers
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import mean_absolute_percentage_error
import joblib
from tensorflow.keras.models import load_model

In [2]:
list_of_10_stocks = ['T', 'INTC', 'ADBE', 'JPM', 'PG', 'NVDA', 'AAPL', 'AMZN', 'UNH', 'MA']
#list_of_10_stocks = ['T', 'INTC']

# Loads model and transformer

In [ ]:
cwd = os.getcwd()
simp_path4 = 'raw_data/df.csv'
abs_path4 = os.path.abspath(os.path.join(cwd,'..',simp_path4))

In [3]:
dict_of_scaler = {}
dict_of_model = {}

for stock in list_of_10_stocks:
    dict_of_scaler[stock] = joblib.load(f'{stock}_scaler.joblib')
    dict_of_model[stock] = load_model(f'{stock}_model.h5')

In [4]:
#dict_of_model.keys()

# Load dataframe

In [5]:
df = get_portfolio_data()

In [6]:
#df.dtypes

In [7]:
#df.head(5)

# Create classes

In [8]:
class Stock:
    def __init__(self, name, initial_price, final_price, dollars):
        self.name = name
        self.initial_price = initial_price
        self.final_price = final_price
        
        self.current_price = initial_price

        self.amount_owned = dollars
        self.num_shares = dollars/initial_price
        self.prediction_higher = True
        
        self.prediction = 0.0
        self.prices = 0.0
    
    def prices(self,number):
        self.prices = number
         

# User input

In [9]:
initial_date = '2015-02-02'
final_date = '2015-12-30'
initial_amount = 10000
in_hand = 0

# Instatiate classes

In [10]:
list_stocks = {}
for i in list_of_10_stocks:
    qty = len(list_of_10_stocks)
    price = df[(df["date"]==initial_date) & (df["ticker"]==i)]
    final_price = df[(df["date"]==final_date) & (df["ticker"]==i)]
    list_stocks[i] = Stock(i, price['adj_close'], final_price['adj_close'], initial_amount/qty)

In [11]:
list_stocks

{'T': <__main__.Stock at 0x7fc9242cb310>,
 'INTC': <__main__.Stock at 0x7fc9242cba90>,
 'ADBE': <__main__.Stock at 0x7fc9242cb4f0>,
 'JPM': <__main__.Stock at 0x7fc9242cb9d0>,
 'PG': <__main__.Stock at 0x7fc9242cb0a0>,
 'NVDA': <__main__.Stock at 0x7fc9242cb580>,
 'AAPL': <__main__.Stock at 0x7fc9242b59d0>,
 'AMZN': <__main__.Stock at 0x7fc9242b5190>,
 'UNH': <__main__.Stock at 0x7fc9242b5310>,
 'MA': <__main__.Stock at 0x7fc9242b53d0>}

# Create predictions

In [12]:
for s in list_stocks:
    df_T2 = df[df['ticker']== s]
    df_T2.reset_index(inplace=True)
    ind = df_T2.index[df_T2['date'] == initial_date].tolist()
    ind2 = df_T2.index[df_T2['date'] == final_date].tolist()
    df_T2 = df_T2[ind[0]-18:ind2[0]+1]
    df_T2 = df_T2.adj_close
    df_T2 = pd.DataFrame(df_T2)
    
    scaler = dict_of_scaler[s]
    X_test_T = scaler.transform(df_T2)
    
    model = dict_of_model[s]
    X_test_T2 = create_sequences_scaled(pd.DataFrame(X_test_T))
    prediction = model.predict(X_test_T2)

    prediction_back = scaler.inverse_transform(prediction.reshape(-1, 1))
    
    list_stocks[s].prediction = prediction_back
    
    df_T2 = df[df['ticker']== s]
    df_T2.reset_index(inplace=True)
    ind = df_T2.index[df_T2['date'] == initial_date].tolist()
    ind2 = df_T2.index[df_T2['date'] == final_date].tolist()
    df_T2 = df_T2[ind[0]:ind2[0]+1]
    list_stocks[s].prices = df_T2

# Return of buy and hold

In [13]:
buyandhold = 0.0
for i in list_stocks:
    #print(list_stocks[i].amount_owned / list_stocks[i].initial_price *float(list_stocks[i].final_price))
    buyandhold+=float(list_stocks[i].amount_owned / list_stocks[i].initial_price *float(list_stocks[i].final_price))

In [14]:
buyandhold

12544.832335311792

# Loop through days using models

In [15]:
df_ = df[df['ticker']== 'T']
df_.reset_index(inplace=True)
ind = df_.index[df_['date'] == initial_date].tolist()
ind2 = df_.index[df_['date'] == final_date].tolist()
df_ = df_[ind[0]:ind2[0]+1]

In [16]:
#loops through days
for i in range(len(df_)-1):
    print(f"-----------------")
    print(f"loop number: {i}")

    #for each day it checks the prediction and appends buy list
    buy_list = []
    
    for s in list_stocks:
        if float(list_stocks[s].prediction[i]) >= float(list_stocks[s].current_price):
            #list_stocks[s].prediction_higher = True
            buy_list.append(s)
        #else: 
            #list_stocks[s].prediction_higher = False

    #sells and puts all the money on wallet
    for s in list_stocks:
        in_hand += float(list_stocks[s].num_shares * list_stocks[s].current_price)
        #list_stocks[s].amount_owned = 0
        list_stocks[s].num_shares = 0
    
    print(f"buy list: {buy_list}")

    #buys
    if len(buy_list) > 0:
        for l in buy_list:
            amount_to_buy = float(in_hand / len(buy_list))

            list_stocks[l].num_shares =  amount_to_buy / float(list_stocks[l].current_price)

        in_hand = 0

    
    owned=0
    for s in list_stocks:
        owned += float(list_stocks[s].num_shares * list_stocks[s].current_price)  

    
    #update current price
    for s in list_stocks:
        list_stocks[s].current_price = list_stocks[s].prices.adj_close.iloc[i]
    
    #print(f"amount in hand after loop: {in_hand}")

-----------------
loop number: 0
buy list: ['INTC']
-----------------
loop number: 1
buy list: ['INTC', 'NVDA']
-----------------
loop number: 2
buy list: ['INTC', 'NVDA']
-----------------
loop number: 3
buy list: ['INTC', 'NVDA']
-----------------
loop number: 4
buy list: ['INTC']
-----------------
loop number: 5
buy list: ['INTC', 'NVDA']
-----------------
loop number: 6
buy list: ['INTC', 'NVDA']
-----------------
loop number: 7
buy list: ['INTC']
-----------------
loop number: 8
buy list: ['INTC', 'NVDA']
-----------------
loop number: 9
buy list: ['INTC', 'NVDA']
-----------------
loop number: 10
buy list: ['INTC', 'NVDA']
-----------------
loop number: 11
buy list: ['INTC']
-----------------
loop number: 12
buy list: ['INTC', 'NVDA']
-----------------
loop number: 13
buy list: ['INTC', 'NVDA']
-----------------
loop number: 14
buy list: ['INTC']
-----------------
loop number: 15
buy list: ['INTC', 'NVDA']
-----------------
loop number: 16
buy list: []
-----------------
loop numb

In [17]:
for s in list_stocks:
    print(f"{list_stocks[s].name}:  {list_stocks[s].current_price}")

T:  30.3335742950439
INTC:  32.7508544921875
ADBE:  95.3300018310547
JPM:  62.3459663391113
PG:  73.5216979980469
NVDA:  33.1886825561523
AAPL:  103.501159667969
AMZN:  693.969970703125
UNH:  115.227798461914
MA:  97.7733764648438


In [18]:
for s in list_stocks:
    print(f"{list_stocks[s].name}:  {list_stocks[s].num_shares}")

T:  0
INTC:  446.0941840412183
ADBE:  0
JPM:  0
PG:  0
NVDA:  440.9442103171836
AAPL:  0
AMZN:  0
UNH:  0
MA:  0


# Checks final amounts

In [19]:
in_hand

0

In [20]:
final_amount = in_hand
for s in list_stocks:
    final_amount += float(list_stocks[s].num_shares * list_stocks[s].current_price)
    

In [21]:
final_amount

29244.323132535315

In [22]:
print(f"Buy and hold strategy made {buyandhold - initial_amount} ({(buyandhold - initial_amount)/initial_amount}%)")
print(f"Our AI trader made {final_amount - initial_amount} ({(final_amount - initial_amount)/initial_amount}%)")

Buy and hold strategy made 2544.832335311792 (0.2544832335311792%)
Our AI trader made 19244.323132535315 (1.9244323132535315%)
